
\begin{center}
Chun-Yuan (Scott) Chiu
\end{center}
\begin{center}
chunyuac@andrew.cmu.edu
\end{center}

# 2. {-}



In [66]:
from pandas import DataFrame
from scipy.special import beta
from scipy.optimize import minimize, root_scalar
from scipy.integrate import quad
from scipy import stats
import numpy as np
import numdifftools as nd

def pdf(x, mu, k, n, lam, sigma2, log=False):
    '''
    k > 0, n > 2, sigma2 > 0, -1 < lam < 1
    '''
    sigma = np.sqrt(sigma2)

    S = np.sqrt(1 + 3*(lam**2) - 4*(lam**2)*((beta(2/k, (n-1)/k)**2)/(beta(1/k, n/k)*beta(3/k, (n-2)/k))))
    c = 0.5*k*np.sqrt((beta(1/k, n/k)**(-3))*beta(3/k, (n-2)/k))*S/sigma
    theta = ((k/(n-2))**(1/k))*np.sqrt(beta(1/k, n/k)/beta(3/k, (n-2)/k))/S
    
    if log:
        return np.log(c)-(n+1)*np.log(1 + (k/(n-2))*(np.abs(x-mu)/(sigma*theta*(1 + np.sign(x-mu)*lam)))**k)/k
    else:
        return c*(1+(k/(n-2))*(np.abs(x-mu)/(sigma*theta*(1 + np.sign(x-mu)*lam)))**k)**(-(n+1)/k)

def cdf(x, mu, k, n, lam, sigma2):
    integrand = lambda u: pdf(u-mu, 0, k, n, lam, sigma2)
    return quad(integrand, 0, x)[0] + (1-lam)/2

def ppf(y, mu, k, n, lam, sigma2):
    assert 0 < y < 1, 'y must be in [0, 1].'  
    res = root_scalar(lambda x: cdf(x, mu, k, n, lam, sigma2)-y, method='newton', x0=mu, fprime=lambda x: pdf(x, mu, k, n, lam, sigma2))
    assert res.converged, "Newton's method failed to converge."
    return res.root

def rvs(size, mu, k, n, lam, sigma2):
    return np.array([ppf(u, mu, k, n, lam, sigma2) for u in np.random.uniform(size=size)])

def fitSkewT(x, allowshift=False):
    
    if(allowshift):
        initpars = [np.mean(x), 2, 15, 2*(np.array(x)>0).sum()/len(x)-1, np.var(x)]
        def negloglikelihood(params, x):
            mu, k, n, lam, sigma2 = params
            return (-1)*pdf(x, mu, k, n, lam, sigma2, log=True).sum()
        
        mleout = minimize(negloglikelihood, initpars, args=(x), method='L-BFGS-B', bounds=[(None, None), (0.001, None), (2.001, None), (-0.999, 0.999), (0.001, None)])
        hessfunc = nd.Hessian(negloglikelihood)
        return mleout.x, np.linalg.inv(hessfunc(mleout.x, x))
    
    else:
        initpars = [2, 15, 2*(x>0).sum()/len(x)-1, np.var(x)]
        def negloglikelihood(params, x):
            k, n, lam, sigma2 = params
            return (-1)*pdf(x, 0, k, n, lam, sigma2, log=True).sum()
        
        mleout = minimize(negloglikelihood, initpars, args=(x), method='L-BFGS-B', bounds=[(0.001, None), (2.001, None), (-0.999, 0.999), (0.001, None)])
        hessfunc = nd.Hessian(negloglikelihood)
        mat = hessfunc(mleout.x, x)
        return mleout.x, np.linalg.inv(hessfunc(mleout.x, x))
    
size = 10000
k = 2
n = 10
lam = -0.5
sigma2 = 1
mu = 0

x = rvs(size, mu, k, n, lam, sigma2)

mle, var = fitSkewT(x, allowshift=False)
z = stats.norm.ppf(1-0.025)

print('MLE for k, n, lambda and sigma2: ', mle)
print()
print('Covariance matrix of the MLE: \n', DataFrame(var, columns=['k', 'n', 'lambda', 'sigma2'], index=['k', 'n', 'lambda', 'sigma2']))
print()
print('Confidence Intervals for the 4 parameters: ', [(mle[i] - z*np.sqrt(var[i][i]), mle[i] + z*np.sqrt(var[i][i])) for i in range(4)])


/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in sqrt
  app.launch_new_instance()
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in double_scalars
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in sqrt
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in sqrt
  from ipykernel import kernelapp as app
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in power
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in sqrt
  del sys.path[0]
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in sqrt

MLE for k, n, lambda and sigma2:  [ 2.06043762  9.38748487 -0.49932169  1.01037844]

Covariance matrix of the MLE: 
                k         n    lambda    sigma2
k       0.010649 -0.139952  0.000006  0.000004
n      -0.139952  2.456909 -0.000057 -0.007943
lambda  0.000006 -0.000057  0.000029 -0.000016
sigma2  0.000004 -0.007943 -0.000016  0.000372

Confidence Intervals for the 4 parameters:  [(1.8581809769961335, 2.2626942606718004), (6.31533327736936, 12.459636457361183), (-0.5099504519759688, -0.48869291880944815), (0.972588995801543, 1.048167890244419)]


In [72]:
size = 50000
k = 2
n = 10
lam = -0.5
sigma2 = 1
mu = 0

x = rvs(size, mu, k, n, lam, sigma2)

mle, var = fitSkewT(x, allowshift=True)
z = stats.norm.ppf(1-0.025)

print('MLE for mu, k, n, lambda and sigma2: ', mle)
print()
print('Covariance matrix of the MLE: \n', DataFrame(var, columns=['mu', 'k', 'n', 'lambda', 'sigma2'], index=['mu', 'k', 'n', 'lambda', 'sigma2']))
print()
print('Confidence Intervals for the 5 parameters: ', [(mle[i] - z*np.sqrt(var[i][i]), mle[i] + z*np.sqrt(var[i][i])) for i in range(5)])


/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in sqrt
  from ipykernel import kernelapp as app
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in double_scalars
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in power
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in sqrt
  del sys.path[0]
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in sqrt
  app.launch_new_instance()
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in sqrt


MLE for mu, k, n, lambda and sigma2:  [ 0.01167648  1.98181326 10.40948959 -0.50982027  0.99923617]

Covariance matrix of the MLE: 
               mu             k         n    lambda        sigma2
mu      0.000062 -3.505471e-06  0.000065 -0.000039  2.067267e-05
k      -0.000004  2.128455e-03 -0.037982  0.000002 -5.190427e-07
n       0.000065 -3.798166e-02  0.868287 -0.000044 -1.872945e-03
lambda -0.000039  2.359622e-06 -0.000044  0.000030 -1.598994e-05
sigma2  0.000021 -5.190427e-07 -0.001873 -0.000016  7.899016e-05

Confidence Intervals for the 5 parameters:  [(-0.0037854687370751094, 0.02713842989314266), (1.8913899620243335, 2.072236555753614), (8.583157591667385, 12.235821580521863), (-0.5205544493562151, -0.4990860848061738), (0.9818167186982341, 1.016655630383658)]
